<a href="https://colab.research.google.com/github/abj-paul/Nonnegative-Matrix-Factorization-Experimentation/blob/main/NMF_with_training_and_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Equations from https://proceedings.neurips.cc/paper/2000/file/f9d1152547c0bde01830b7e8bd60024c-Paper.pdf

In [99]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from sklearn.cluster import KMeans
import numpy
import matplotlib.pyplot as mtp
import os

# Code Library

In [131]:
import time

def extractFrameFromImage(filePath, imageLimit, frame_rate):
  vidcap = cv2.VideoCapture(filePath)
  success,image = vidcap.read()
  count = 0
  #define framerate
  prev = 0
  frame=0
  while success:
    time_elapsed = time.time() - prev
    success,image = vidcap.read()  
    if time_elapsed > 1./frame_rate:
      prev = time.time()
      cv2.imwrite("%d.jpg" % count, image)     # save frame as JPEG file      
      #print('Read a new frame: ', success)
      count=count+1
      frame += frame_rate # i.e. at 30 fps, this advances one second
      vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame)
    if count>=imageLimit:
      break
  vidcap.release()
  cv2.destroyAllWindows()

  

In [152]:
def countWidthandHeight(imagePath, downscalingFactor):

  for img in os.listdir(imagePath):
    extension = os.path.splitext(img)[1][1:]
    if extension=='jpg':
      sampleImagePath=imagePath+'/'+img
      img=cv2.imread(sampleImagePath)
      image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      #resize

      width = int(image.shape[1]/downscalingFactor)
      height = int(image.shape[0]/downscalingFactor)

      print(width, height)

      break


  count=0

  for image_name in os.listdir(imagePath):
    extension = os.path.splitext(image_name)[1][1:]
    if extension=='jpg':
      count=count+1

  return width, height, count

In [102]:
def convertAllImageToMatrix(width, height, imageCount, imageRootDir): # imageCount = number of images  , imageRootDir = "/content"

  y = width*height 

  m = np.zeros(shape=(imageCount, int(y)))
  print(m.shape)
  itr = 0
  name_list = []
  for image_name in os.listdir(imageRootDir): 
    extension = os.path.splitext(image_name)[1][1:]
    if extension=='jpg' or extension=='png':
      image_path=imageRootDir+'/'+image_name
      image = cv2.imread(image_path)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      #resize
      dim = (width, height)
      print(width, height)
      #resizing image
      resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
      
      rows,cols=resized.shape#normal shape
      img_size = rows*cols
      img_1D_vector = resized.reshape(img_size)
      
      m[itr]=img_1D_vector
      itr=itr+1
      name_list.append(image_name)
      print(itr)
      
  return m, name_list, itr

In [103]:
def normalizeM(m):
  sum=0
  row, col=m.shape
  for i in range(0,row):
    for j in range(0,col):
      sum+=m[i][j]
  mean=sum/row*col 

  m=m/mean

  return m, mean

In [104]:
def updateW(x,w,h):
  #XVT
  ht=np.transpose(h)
  numerator=np.matmul(x,ht)
  #UVVT
  den1=np.matmul(w,h)
  denominator=np.matmul(den1,ht)

  #return
  print('U completed#########################################################################################')
  secondPart=np.divide(numerator,denominator)
  return np.multiply(w,secondPart)

In [105]:
def updateH(x,w,h):
  #UT*X
  wt=np.transpose(w)
  numerator=np.matmul(wt,x)
  #UTUV
  den1=np.matmul(w,h)
  denominator=np.matmul(wt,den1)
  
  #update return
  print('V completed#########################################################################################')
  secondPart=np.divide(numerator,denominator)
  return np.multiply(h,secondPart)

In [106]:
def performNMF(x, featureNo, imageCount, EPSILON=1e-9):
  w=np.random.normal(3,2.5,size=(imageCount,featureNo))
  h=np.random.normal(3,2.5,size=(featureNo, x.shape[1]))

  print("Shape of W: "+str(w.shape))
  print("Shape of H: "+str(h.shape))

  itr=0
  cost=np.linalg.norm(x-np.matmul(w,h))
  while cost>EPSILON:
    if itr%2==0:
      h=updateH(x,w,h)
    else:
      w=updateW(x,w,h)
    wh=np.matmul(w,h)
    print(w,h)
    cost=np.linalg.norm(x-wh)
    print(cost)
    if itr>=100:
      break
    itr=itr+1
  return w,h

### Testing Code

In [107]:
from google.colab.patches import cv2_imshow

In [108]:
def getImageListFromM(x_new):
  images = []
  for row in x_new:
    img_data = []
    for i in range(height):
      img_row = []
      for j in range(width):
        img_row.append(row[i*width + j])
      img_data.append(img_row)
    images.append(img_data)
  return images

In [109]:
def compareAndTest(m, w, h, mean):
  x_new=np.matmul(w,h)
  x_new=np.ceil(x_new*mean)
  #x_new=np.transpose(x_new)

  resulting_images = getImageListFromM(x_new)
  original_images = getImageListFromM(np.ceil(m*mean))

  for index in range(m.shape[0]):
    print("-------------------------------Number of image: "+str(index))

    img=np.array(resulting_images[index])
    cv2_imshow(img)
    print("Image after NMF:")

    img=np.array(original_images[index])
    cv2_imshow(img)
    print("Image Before NMF:")


In [110]:
def variance(m): # m = matrix of all images
  return np.var(m)

In [111]:
def __grouper(labels, original_images, resulting_images):
    groups = []
    for current_label in set(labels):
      cluster = []
      for index,label in enumerate(labels):
        if label == current_label:
          node = [label, original_images[index], resulting_images[index]]
          cluster.append(node)
      groups.append(cluster)
    return groups

In [112]:
def compareAndTestWithKNN(m, w, h, mean, n_clusters):
  x_new=np.matmul(w,h)
  x_new=np.ceil(x_new*mean)
  #x_new=np.transpose(x_new)

  kmeans = KMeans(n_clusters, random_state=0)
  kmeans.fit(w)

  resulting_images = getImageListFromM(x_new)
  original_images = getImageListFromM(np.ceil(m*mean))
  
  clusters = __grouper(kmeans.labels_, original_images, resulting_images)

  for cluster in clusters:
    print("----------------Labels------------------------")
    for index, node in enumerate(cluster):
      print("-------------------------------Number of image: "+str(index)+"-----------Cluster="+str(node[0])+".")
      img=np.array(node[1])
      cv2_imshow(img)
  
  return kmeans

In [113]:
def scatterPlot(kmeans,F):
  y_predict= kmeans.fit_predict(F)  
  mtp.scatter(F[y_predict == 0, 0], F[y_predict == 0, 1], s = 10, c = 'blue', label = 'Cluster 1') #for first cluster  
  mtp.scatter(F[y_predict == 1, 0], F[y_predict == 1, 1], s = 10, c = 'green', label = 'Cluster 2') #for second cluster  
  mtp.scatter(F[y_predict== 2, 0], F[y_predict == 2, 1], s = 10, c = 'red', label = 'Cluster 3') #for third cluster  
  mtp.scatter(F[y_predict == 3, 0], F[y_predict == 3, 1], s = 10, c = 'cyan', label = 'Cluster 4') #for fourth cluster  
  mtp.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 30, c = 'yellow', label = 'Centroid')   
  mtp.title('Clusters')  

  mtp.legend()  
  mtp.show() 

# Experimentation Area

In [114]:
def getSingleImageFromMatrix(data, width, height):
  img_data = []
  for i in range(height):
    img_row = []
    for j in range(width):
      img_row.append(data[i*width + j])
    img_data.append(img_row)
  return img_data

In [115]:
imageLimit = 100
frame_rate=5
extractFrameFromImage('/content/Drive/MyDrive/Cocurriculur/RMGProject/Dataset/ClothStackDetection.mp4', imageLimit, frame_rate)

In [116]:
!ls

0.jpg	20.jpg	31.jpg	42.jpg	53.jpg	64.jpg	75.jpg	86.jpg	97.jpg
10.jpg	21.jpg	32.jpg	43.jpg	54.jpg	65.jpg	76.jpg	87.jpg	98.jpg
11.jpg	22.jpg	33.jpg	44.jpg	55.jpg	66.jpg	77.jpg	88.jpg	99.jpg
12.jpg	23.jpg	34.jpg	45.jpg	56.jpg	67.jpg	78.jpg	89.jpg	9.jpg
13.jpg	24.jpg	35.jpg	46.jpg	57.jpg	68.jpg	79.jpg	8.jpg	Drive
14.jpg	25.jpg	36.jpg	47.jpg	58.jpg	69.jpg	7.jpg	90.jpg	sample_data
15.jpg	26.jpg	37.jpg	48.jpg	59.jpg	6.jpg	80.jpg	91.jpg
16.jpg	27.jpg	38.jpg	49.jpg	5.jpg	70.jpg	81.jpg	92.jpg
17.jpg	28.jpg	39.jpg	4.jpg	60.jpg	71.jpg	82.jpg	93.jpg
18.jpg	29.jpg	3.jpg	50.jpg	61.jpg	72.jpg	83.jpg	94.jpg
19.jpg	2.jpg	40.jpg	51.jpg	62.jpg	73.jpg	84.jpg	95.jpg
1.jpg	30.jpg	41.jpg	52.jpg	63.jpg	74.jpg	85.jpg	96.jpg


In [117]:
width, height, imageCount=countWidthandHeight("/content", 7)

384 217


In [118]:
print(imageCount)

100


In [ ]:
m, imageNameList, imageCount = convertAllImageToMatrix(width, height, imageCount, "/content")

In [120]:
m.shape

(100, 83328)

In [121]:
m, mean = normalizeM(m)

In [122]:
w,h=performNMF(m,10, len(imageNameList))
'''
from sklearn.decomposition import NMF
nmf_model = NMF(10,max_iter=100)
w = nmf_model.fit_transform(m)
h=nmf_model.components_
'''

Streaming output truncated to the last 5000 lines.
  -2.05981539e-21  9.81141984e-19]
 [ 8.04260343e-11 -1.92830873e-30  5.12518981e-27 -1.91754576e-18
   3.75891958e-17 -3.74391060e-28  5.38152692e-20  2.58492765e-19
  -2.47800117e-21 -3.15558253e-19]
 [ 8.67700968e-11 -1.83078496e-30 -2.13729168e-24  3.13205518e-23
   5.13737743e-17  1.87244187e-21  1.17601142e-17  3.76092331e-19
  -3.01429172e-21  2.11078435e-18]
 [ 8.78105510e-11  3.75267057e-27  7.70317312e-28 -2.15238938e-21
   8.02795839e-17  1.48973077e-26 -5.87085543e-20  5.84122392e-17
  -6.54877473e-20  8.05801926e-20]
 [ 8.54432879e-11  3.44905579e-27 -6.60923848e-25 -2.31392102e-22
   6.71462785e-17  4.85292152e-24 -1.34486434e-19  5.61295545e-17
  -6.29531746e-20  1.83483693e-18]
 [ 8.02846243e-11 -8.31186550e-31 -9.89442275e-23  3.25225102e-20
   3.10158337e-17 -2.18654147e-22  3.20825131e-17  1.17060755e-19
  -3.88815185e-22  2.40689811e-18]
 [ 8.44150748e-11  2.98019935e-31 -6.42388582e-29  8.67865879e-20
   3.92619099

'\nfrom sklearn.decomposition import NMF\nnmf_model = NMF(10,max_iter=100)\nw = nmf_model.fit_transform(m)\nh=nmf_model.components_\n'

In [123]:
#compareAndTest(m,w,h, mean)

In [124]:
x_new=np.matmul(w,h)
x_new=np.ceil(x_new*mean)
x_new=np.transpose(x_new)

print("Original Image Set Variance: "+str(variance(np.ceil(np.transpose(m)*mean))))
print("NMF Generated Image Set Variance: "+str(variance(x_new)))

Original Image Set Variance: 4066.2712745431204
NMF Generated Image Set Variance: 11310.538269079507


In [125]:
w.shape

(100, 10)

# Clustering

In [126]:
#kmeans=compareAndTestWithKNN(m,w,h, mean, 3)

In [127]:
#scatterPlot(kmeans,w)

In [128]:
test=m[0,0:]
test_vector = np.array(test)[np.newaxis]
print(test_vector.shape)
print(np.transpose(h).shape)
kmeans.predict(np.matmul(test_vector, np.transpose(h)))

(1, 83328)
(83328, 10)


array([1], dtype=int32)

In [129]:
import math
def testSingleImage(image_path, width, height,h):
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  dim = (width, height)
  resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  
  rows,cols=resized.shape
  img_size = rows*cols
  img_1D_vector = resized.reshape(img_size)

  img_1D_matrix = np.array(img_1D_vector)[np.newaxis]
  return kmeans.predict(np.matmul(img_1D_matrix, np.transpose(h)))[0]


In [161]:
def testSingleFrameMatrix(image, width, height, kmeans, h):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  dim = (width, height)
  resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  
  rows,cols=resized.shape
  img_size = rows*cols
  img_1D_vector = resized.reshape(img_size)

  img_1D_matrix = np.array(img_1D_vector)[np.newaxis]
  return kmeans.predict(np.matmul(img_1D_matrix, np.transpose(h)))[0]


In [159]:
from statistics import mode
def testVideo(videoFilePath, imageLimit, frame_rate, width, height, kmeans, h):
  vidcap = cv2.VideoCapture(videoFilePath)
  success,image = vidcap.read()
  count = 0
  #define framerate
  prev = 0
  frame=0
  results = []
  while success:
    time_elapsed = time.time() - prev
    success,image = vidcap.read()  

    frame += frame_rate # i.e. at 30 fps, this advances one second
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame)

    dim = (width, height)
    resized_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    results.append(testSingleFrameMatrix(resized_image, width, height, kmeans, h))

    print(results[count])
    if count>imageLimit: break
    count += 1
  return mode(results)

In [154]:
def train(path,n_components, n_clusters):
  imageLimit = 100
  frame_rate=5
  #extractFrameFromImage(path, imageLimit, frame_rate)
  width, height, imageCount=countWidthandHeight("/content", 7)
  m, imageNameList, imageCount = convertAllImageToMatrix(width, height, imageCount, "/content")
  m, mean = normalizeM(m)
  w,h=performNMF(m, n_components, len(imageNameList))
  x_new=np.matmul(w,h)
  x_new=np.ceil(x_new*mean)
  x_new=np.transpose(x_new)
  print("Original Image Set Variance: "+str(variance(np.ceil(np.transpose(m)*mean))))
  print("NMF Generated Image Set Variance: "+str(variance(x_new)))
  
  kmeans = KMeans(n_clusters, random_state=0)
  kmeans.fit(w)

  return m, w, h, kmeans

In [163]:
m,w,h,kmeans=train('/content/Drive/MyDrive/Cocurriculur/RMGProject/Dataset/trainingabhi.mp4',10,2)

Streaming output truncated to the last 5000 lines.
   1.34590567e-38  1.27807544e-16  1.03576803e-37 -2.84598571e-35
   7.84274864e-34  5.87755941e-38]
 [ 5.45816903e-35 -1.11250892e-41 -9.23590271e-35 -1.40056034e-34
   9.23161972e-39  1.27783469e-16  1.56874764e-36 -2.84707501e-35
   7.80197275e-34  4.42968105e-38]
 [-1.08849766e-36  1.14041564e-40 -1.11244547e-34 -2.93169114e-35
  -4.21791131e-36  1.41387366e-16  7.18196737e-41 -3.40896955e-35
   9.45279554e-34  1.68831485e-36]
 [ 6.96984898e-34  1.04270982e-40 -1.12644455e-34 -1.98027679e-34
   2.52461838e-37  1.40302207e-16 -1.28763010e-35 -3.54287535e-35
   9.14418976e-34  1.53810134e-36]
 [ 5.66322363e-34 -4.30650457e-38 -8.94173582e-35 -1.32996238e-34
   1.46596992e-37  1.26059896e-16  4.09011099e-35 -6.67319636e-39
   7.45297853e-34  4.14512625e-40]
 [-2.46812431e-35  6.31830253e-40 -1.06523718e-34 -3.59003697e-35
  -2.47688765e-41  1.25854268e-16 -4.13950217e-43 -3.02809101e-35
   8.60268529e-34  5.99188183e-37]
 [-6.78193434

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [162]:
testVideo("/content/Drive/MyDrive/Cocurriculur/RMGProject/Dataset/testabhi.mp4", 100, 30, width, height,kmeans,h)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0